<a href="https://colab.research.google.com/github/wilfredosanchez/ai-product-incubator/blob/main/AI_Product_Incubator_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 AI PRODUCT INCUBATOR v1.0

**Welcome! Let's generate 15 Etsy products in 30 seconds.**

---

### **STEP 1: Enter Your API Key**
🔑 You'll need a free OpenAI API key.  
👉 Get it here: **platform.openai.com/api-keys**  
They give you $5 free credit—enough for 250+ uses.

### **STEP 2: Run the WIZARD**
Click the ▶️ play button below, then answer 4 simple questions.

### **STEP 3: Get Your Products**
AI will generate 15 product ideas with names, keywords, and profit margins.

---

**Ready? Click play below! ⬇️**

In [13]:
# @title
# 🚀 RUN THIS CELL FIRST
# Click the play button ▶️ to the left

print("✅ STEP 1: Clicked play!")
print("Now scroll down and run the main incubator cell.")
print("="*50)


✅ STEP 1: Clicked play!
Now scroll down and run the main incubator cell.


In [ ]:
# @title
# AI PRODUCT INCUBATOR v2.0 - UPDATED FOR OPENAI 1.0+
# Compatible with latest openai-python library

# Install latest OpenAI (run this once)
!pip install --upgrade openai -q

import openai
import os

# Get user's API key
print("🚀 AI PRODUCT INCUBATOR v2.0")
print("="*50)
print("🔑 Get free API key at: platform.openai.com/api-keys")
print("They give $5 free credit = 250+ uses")
print("="*50)

API_KEY = input("Paste your OpenAI API key: ")

# Set up OpenAI client (new syntax)
client = openai.OpenAI(api_key=API_KEY)



---

### **🎯 NOW LET'S GET READY TO BUILD YOUR PRODUCTS**

Prepare to answer these 4 questions (type your answer, press Enter):

1. What do you sell? (e.g., printable wall art)
2. Who's your buyer? (e.g., new moms)
3. Price range? (e.g., $10-25)
4. Competitor name? (or type 'none')

**Remember...AI is thinking... this takes 10 seconds.**

**Ready? Click play below! ⬇️**

In [ ]:
# @title
# Get user input
niche = input("\nWhat do you sell? (e.g., printable wall art): ")
audience = input("Who's your buyer? (e.g., new moms): ")
price_range = input("Price range? (e.g., $10-25): ")
competitors = input("Competitor shop name (or 'none'): ")

# Build the master prompt
prompt = f"""You are an Etsy AI strategist with 10+ years experience. Generate 15 hyper-specific product ideas for:
- NICHE: {niche}
- TARGET AUDIENCE: {audience}
- PRICE RANGE: {price_range}
- COMPETITOR AWARENESS: {competitors}

For each product, provide:
1. Product name (under 55 characters for Etsy SEO)
2. 5 high-traffic, low-competition keywords
3. Production difficulty (1-10 scale)
4. Trend score (1-10, based on 2025 data)
5. Estimated profit margin
6. One-sentence description

Format as a markdown table. Be specific, actionable, and data-driven."""

try:
    # New API syntax
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    print("\n✨ YOUR 15 PRODUCT IDEAS:\n")
    print(response.choices[0].message.content)

    print("\n" + "="*50)
    print("💡 Save this output! Run again anytime for fresh ideas.")

except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("💡 Fix: Check your API key is copied correctly")

---

### ✅ **DONE! HERE'S WHAT TO DO NEXT:**

1. **Copy your favorite product names** from above
2. **Use the keywords** in your Etsy tags
3. **Check the profit margin** before creating
4. **Run again anytime** for fresh ideas (click Runtime → Run all)

**Questions? Email: your-email@example.com**

**Happy selling! 🎉**